In [1]:
import torch
from torch import nn,optim
import functools
import matplotlib.pyplot as plt
from torch.autograd import Variable
import torchvision.utils as vutils

In [2]:
import random
import numpy as np
import torch.utils.data as data
from PIL import Image
import torchvision.transforms as transforms

In [45]:
from tensorboardX import SummaryWriter
writer=SummaryWriter('/user1/faculty/cvpr/ujjwal/dhritimaan/GANs/datasets/log_cycle_gan/')

In [46]:
class SequenceFolder(data.Dataset):
    def __init__(self, trainA,trainB):
        self.trainA=trainA
        self.trainB=trainB

    def __len__(self):
        return 400
    def __getitem__(self, index):
        imgA=plt.imread(self.trainA[index])
        imgB=plt.imread(self.trainB[index])
        dict_img={'A':imgA,'B':imgB}
        return dict_img

In [47]:
import os
a=os.listdir('/user1/faculty/cvpr/ujjwal/dhritimaan/GANs/datasets/facades/trainA')
b=os.listdir('/user1/faculty/cvpr/ujjwal/dhritimaan/GANs/datasets/facades/trainB')
lista=[]
listb=[]
for i in range(400):
    lista.append(os.path.join('/user1/faculty/cvpr/ujjwal/dhritimaan/GANs/datasets/facades/trainA',a[i]))
    listb.append(os.path.join('/user1/faculty/cvpr/ujjwal/dhritimaan/GANs/datasets/facades/trainB',b[i]))

In [48]:
train=SequenceFolder(lista,listb)

In [49]:
dataloader=torch.utils.data.DataLoader(train,batch_size=4,shuffle=True)

In [50]:
import matplotlib.pyplot as plt
for a,b in enumerate(dataloader):
    a
    break

In [51]:
b['A'].shape

torch.Size([4, 256, 256, 3])

In [52]:
b['B'].permute(0,3,1,2).shape

torch.Size([4, 3, 256, 256])

In [53]:
class ResnetBlock(nn.Module):
    """Define a Resnet block"""

    def __init__(self, dim, padding_type, norm_layer, use_dropout, use_bias):
        """Initialize the Resnet block
        A resnet block is a conv block with skip connections
        We construct a conv block with build_conv_block function,
        and implement skip connections in <forward> function.
        Original Resnet paper: https://arxiv.org/pdf/1512.03385.pdf
        """
        super(ResnetBlock, self).__init__()
        self.conv_block = self.build_conv_block(dim, padding_type, norm_layer, use_dropout, use_bias)

    def build_conv_block(self, dim, padding_type, norm_layer, use_dropout, use_bias):
        """Construct a convolutional block.
        Parameters:
            dim (int)           -- the number of channels in the conv layer.
            padding_type (str)  -- the name of padding layer: reflect | replicate | zero
            norm_layer          -- normalization layer
            use_dropout (bool)  -- if use dropout layers.
            use_bias (bool)     -- if the conv layer uses bias or not
        Returns a conv block (with a conv layer, a normalization layer, and a non-linearity layer (ReLU))
        """
        conv_block = []
        p = 0
        if padding_type == 'reflect':
            conv_block += [nn.ReflectionPad2d(1)]
        elif padding_type == 'replicate':
            conv_block += [nn.ReplicationPad2d(1)]
        elif padding_type == 'zero':
            p = 1
        else:
            raise NotImplementedError('padding [%s] is not implemented' % padding_type)

        conv_block += [nn.Conv2d(dim, dim, kernel_size=3, padding=p, bias=use_bias), norm_layer(dim), nn.ReLU(True)]
        if use_dropout:
            conv_block += [nn.Dropout(0.5)]

        p = 0
        if padding_type == 'reflect':
            conv_block += [nn.ReflectionPad2d(1)]
        elif padding_type == 'replicate':
            conv_block += [nn.ReplicationPad2d(1)]
        elif padding_type == 'zero':
            p = 1
        else:
            raise NotImplementedError('padding [%s] is not implemented' % padding_type)
        conv_block += [nn.Conv2d(dim, dim, kernel_size=3, padding=p, bias=use_bias), norm_layer(dim)]

        return nn.Sequential(*conv_block)

    def forward(self, x):
        """Forward function (with skip connections)"""
        out = x + self.conv_block(x)  # add skip connections
        return out

In [54]:
class ResnetGenerator(nn.Module):
    """Resnet-based generator that consists of Resnet blocks between a few downsampling/upsampling operations.
    We adapt Torch code and idea from Justin Johnson's neural style transfer project(https://github.com/jcjohnson/fast-neural-style)
    """

    def __init__(self, input_nc, output_nc, ngf=64, norm_layer=nn.BatchNorm2d, use_dropout=False, n_blocks=6, padding_type='reflect'):
        """Construct a Resnet-based generator
        Parameters:
            input_nc (int)      -- the number of channels in input images
            output_nc (int)     -- the number of channels in output images
            ngf (int)           -- the number of filters in the last conv layer
            norm_layer          -- normalization layer
            use_dropout (bool)  -- if use dropout layers
            n_blocks (int)      -- the number of ResNet blocks
            padding_type (str)  -- the name of padding layer in conv layers: reflect | replicate | zero
        """
        assert(n_blocks >= 0)
        super(ResnetGenerator, self).__init__()
        if type(norm_layer) == functools.partial:
            use_bias = norm_layer.func == nn.InstanceNorm2d
        else:
            use_bias = norm_layer == nn.InstanceNorm2d

        model = [nn.ReflectionPad2d(3),
                 nn.Conv2d(input_nc, ngf, kernel_size=7, padding=0, bias=use_bias),
                 norm_layer(ngf),
                 nn.ReLU(True)]

        n_downsampling = 2
        for i in range(n_downsampling):  # add downsampling layers
            mult = 2 ** i
            model += [nn.Conv2d(ngf * mult, ngf * mult * 2, kernel_size=3, stride=2, padding=1, bias=use_bias),
                      norm_layer(ngf * mult * 2),
                      nn.ReLU(True)]

        mult = 2 ** n_downsampling
        for i in range(n_blocks):       # add ResNet blocks

            model += [ResnetBlock(ngf * mult, padding_type=padding_type, norm_layer=norm_layer, use_dropout=use_dropout, use_bias=use_bias)]

        for i in range(n_downsampling):  # add upsampling layers
            mult = 2 ** (n_downsampling - i)
            model += [nn.ConvTranspose2d(ngf * mult, int(ngf * mult / 2),
                                         kernel_size=3, stride=2,
                                         padding=1, output_padding=1,
                                         bias=use_bias),
                      norm_layer(int(ngf * mult / 2)),
                      nn.ReLU(True)]
        model += [nn.ReflectionPad2d(3)]
        model += [nn.Conv2d(ngf, output_nc, kernel_size=7, padding=0)]
        model += [nn.Tanh()]

        self.model = nn.Sequential(*model)

    def forward(self, input):
        """Standard forward"""
        return self.model(input)

In [55]:
class NLayerDiscriminator(nn.Module):
    """Defines a PatchGAN discriminator"""

    def __init__(self, input_nc, ndf=64, n_layers=3, norm_layer=nn.BatchNorm2d):
        """Construct a PatchGAN discriminator
        Parameters:
            input_nc (int)  -- the number of channels in input images
            ndf (int)       -- the number of filters in the last conv layer
            n_layers (int)  -- the number of conv layers in the discriminator
            norm_layer      -- normalization layer
        """
        super(NLayerDiscriminator, self).__init__()
        if type(norm_layer) == functools.partial:  # no need to use bias as BatchNorm2d has affine parameters
            use_bias = norm_layer.func == nn.InstanceNorm2d
        else:
            use_bias = norm_layer == nn.InstanceNorm2d

        kw = 4
        padw = 1
        sequence = [nn.Conv2d(input_nc, ndf, kernel_size=kw, stride=2, padding=padw), nn.LeakyReLU(0.2, True)]
        nf_mult = 1
        nf_mult_prev = 1
        for n in range(1, n_layers):  # gradually increase the number of filters
            nf_mult_prev = nf_mult
            nf_mult = min(2 ** n, 8)
            sequence += [
                nn.Conv2d(ndf * nf_mult_prev, ndf * nf_mult, kernel_size=kw, stride=2, padding=padw, bias=use_bias),
                norm_layer(ndf * nf_mult),
                nn.LeakyReLU(0.2, True)
            ]

        nf_mult_prev = nf_mult
        nf_mult = min(2 ** n_layers, 8)
        sequence += [
            nn.Conv2d(ndf * nf_mult_prev, ndf * nf_mult, kernel_size=kw, stride=1, padding=padw, bias=use_bias),
            norm_layer(ndf * nf_mult),
            nn.LeakyReLU(0.2, True)
        ]

        sequence += [nn.Conv2d(ndf * nf_mult, 1, kernel_size=kw, stride=1, padding=padw)]  # output 1 channel prediction map
        self.model = nn.Sequential(*sequence)

    def forward(self, input):
        """Standard forward."""
        return self.model(input)

In [56]:
cuda=torch.device('cuda:1')
genAB=ResnetGenerator(3,3).to(cuda)
genBA=ResnetGenerator(3,3).to(cuda)

In [57]:
discAB=NLayerDiscriminator(3,3).to(cuda)
discBA=NLayerDiscriminator(3,3).to(cuda)

In [58]:
from itertools import chain

In [59]:
gen_optim=optim.Adam(chain(genAB.parameters(),genBA.parameters()),lr=0.0002,betas=(0.5,0.999))
dis_optim=optim.Adam(chain(discAB.parameters(),discBA.parameters()),lr=0.0002,betas=(0.5,0.999))

In [60]:
#loss_ce=nn.NLLLoss()
#loss_mse=nn.MSELoss()
#loss_rec=l1loss()

In [61]:
def l1loss(x,y):
    return torch.mean(torch.abs(x-y))
def loss_mse(x,y):
    return torch.mean((x-y)**2)

In [62]:
#nn.L1Loss()

In [ ]:
num_epochs=100
count=0
for epoch in range(num_epochs):
    for n_batches,real_data in enumerate(dataloader):
        realA=real_data['A'].permute(0,3,1,2).float().to(cuda)/255
        realB=real_data['B'].permute(0,3,1,2).float().to(cuda)/255
        
        
        fakeB=genAB(realA)
        fakeA=genBA(realB)
        recA=genBA(fakeB)
        recB=genAB(fakeA)
        dis_real_labB=discAB(realB)
        dis_fake_labB=discAB(fakeB)
        dis_real_labA=discBA(realA)
        dis_fake_labA=discBA(fakeA)
        dis_rec_labA=discBA(recA)
        dis_rec_labB=discAB(recB)
        
        gen_optim.zero_grad()
        #######training generators
        error_genAB=loss_mse(dis_fake_labB,Variable(torch.ones(realA.shape[0],1,30,30).to(cuda)))
        error_genBA=loss_mse(dis_fake_labA,Variable(torch.ones(realA.shape[0],1,30,30).to(cuda)))
        rec_errorA=l1loss(recA,realA)#loss_rec(recA,realA)
        rec_errorB=l1loss(recB,realB)#loss_rec(recB,realB)
                            
        g_loss=error_genAB+error_genBA+20*rec_errorA+20*rec_errorB
        #error_genAB.backward(retain_graph=True)
        #error_genBA.backward()
        #rec_errorA.backward()
        #rec_errorB.backward()
        g_loss.backward()#retain_graph=True)
        
        gen_optim.step()
                            
                            
        #########training discrminators
        dis_optim.zero_grad()
        
        
        fakeB=genAB(realA)
        fakeA=genBA(realB)
        recA=genBA(fakeB)
        recB=genAB(fakeA)
        dis_real_labB=discAB(realB)
        dis_fake_labB=discAB(fakeB)
        dis_real_labA=discBA(realA)
        dis_fake_labA=discBA(fakeA)
        dis_rec_labA=discBA(recA)
        dis_rec_labB=discAB(recB)

        
        
        rec_errorA=loss_mse(dis_real_labB,Variable(torch.ones(realA.shape[0],1,30,30).to(cuda)))
        error_discAB_with_realB=loss_mse(dis_real_labB,Variable(torch.zeros(realA.shape[0],1,30,30).to(cuda)))
        error_discAB_with_fakeB=loss_mse(dis_fake_labB,Variable(torch.zeros(realA.shape[0],1,30,30).to(cuda)))
        error_discBA_with_realB=loss_mse(dis_real_labA,Variable(torch.ones(realA.shape[0],1,30,30).to(cuda)))
        error_discBA_with_realB=loss_mse(dis_fake_labA,Variable(torch.zeros(realA.shape[0],1,30,30).to(cuda)))
        
        disc_loss=error_discAB_with_realB+error_discAB_with_fakeB+error_discBA_with_realB+error_discBA_with_realB
        disc_loss.backward()
        #error_discAB_with_fakeB.backward()
        #error_discBA_with_realB.backward()
        #error_discBA_with_realB.backward()
        if count%10==0:
            writer.add_image("realA",vutils.make_grid(realA,padding=2,normalize=True),count)
            writer.add_image("realB",vutils.make_grid(realB,padding=2,normalize=True),count)
            writer.add_image("fakeA",vutils.make_grid(fakeA.view(fakeA.shape[0]),padding=2,normalize=True),count)
            writer.add_image("fakeB",vutils.make_grid(fakeB,padding=2,normalize=True),count)
            writer.add_image("recA",vutils.make_grid(recA,padding=2,normalize=True),count)
            writer.add_image("recB",vutils.make_grid(recB,padding=2,normalize=True),count)
        
        dis_optim.step()
        
        count=count+1
        print("epoch:{}  disc_loss:{}  gen_loss:{} recA Loss:{} recB Loss:{}".format(epoch,disc_loss,g_loss,rec_errorA
                                                                                     ,rec_errorB))

epoch:0  disc_loss:0.657579243183136  gen_loss:24.917217254638672 recA Loss:0.8229697346687317 recB Loss:0.5846944451332092
epoch:0  disc_loss:0.587417483329773  gen_loss:17.47452163696289 recA Loss:0.8506605625152588 recB Loss:0.4391506314277649
epoch:0  disc_loss:0.5253570079803467  gen_loss:13.311656951904297 recA Loss:0.891198456287384 recB Loss:0.35343021154403687
epoch:0  disc_loss:0.42379483580589294  gen_loss:11.720146179199219 recA Loss:0.8868804574012756 recB Loss:0.2952173948287964
epoch:0  disc_loss:0.37030428647994995  gen_loss:10.022720336914062 recA Loss:0.9036351442337036 recB Loss:0.250381201505661
epoch:0  disc_loss:0.3405129909515381  gen_loss:9.80846881866455 recA Loss:0.9597069025039673 recB Loss:0.23879751563072205
epoch:0  disc_loss:0.3320743441581726  gen_loss:10.110002517700195 recA Loss:0.9626301527023315 recB Loss:0.2594616711139679
epoch:0  disc_loss:0.30445924401283264  gen_loss:8.958944320678711 recA Loss:1.003706932067871 recB Loss:0.1974547952413559
epoc

epoch:0  disc_loss:0.05575686693191528  gen_loss:5.542117118835449 recA Loss:1.0588988065719604 recB Loss:0.09231182187795639
epoch:0  disc_loss:0.04590648040175438  gen_loss:5.826559066772461 recA Loss:1.0346274375915527 recB Loss:0.09491991251707077
epoch:0  disc_loss:0.05688507854938507  gen_loss:5.96302604675293 recA Loss:1.0490026473999023 recB Loss:0.1127542182803154
epoch:0  disc_loss:0.04481648653745651  gen_loss:5.899209976196289 recA Loss:1.0586169958114624 recB Loss:0.09704550355672836
epoch:0  disc_loss:0.036580491811037064  gen_loss:5.280946731567383 recA Loss:1.0578445196151733 recB Loss:0.08997762203216553
epoch:0  disc_loss:0.04995495080947876  gen_loss:5.420129299163818 recA Loss:1.0437637567520142 recB Loss:0.07822389155626297
epoch:0  disc_loss:0.04017842188477516  gen_loss:5.800753593444824 recA Loss:1.0438357591629028 recB Loss:0.08848973363637924
epoch:0  disc_loss:0.03752864897251129  gen_loss:5.112937927246094 recA Loss:1.0494065284729004 recB Loss:0.06589595228

epoch:1  disc_loss:0.018598351627588272  gen_loss:5.044215202331543 recA Loss:1.0291845798492432 recB Loss:0.05658542737364769
epoch:1  disc_loss:0.06438451260328293  gen_loss:5.096673965454102 recA Loss:0.9986088275909424 recB Loss:0.08230998367071152
epoch:1  disc_loss:0.024456745013594627  gen_loss:5.68140983581543 recA Loss:1.0266366004943848 recB Loss:0.08128055185079575
epoch:1  disc_loss:0.02297811768949032  gen_loss:5.094234466552734 recA Loss:1.0214917659759521 recB Loss:0.08140739053487778
epoch:1  disc_loss:0.018566919490695  gen_loss:5.073697090148926 recA Loss:1.0170189142227173 recB Loss:0.06445210427045822
epoch:1  disc_loss:0.03241299092769623  gen_loss:4.546093463897705 recA Loss:1.0295071601867676 recB Loss:0.062337130308151245
epoch:1  disc_loss:0.05032477155327797  gen_loss:5.649947166442871 recA Loss:1.0408493280410767 recB Loss:0.06253308802843094
epoch:1  disc_loss:0.018208706751465797  gen_loss:4.754571914672852 recA Loss:1.0160096883773804 recB Loss:0.075413122

epoch:1  disc_loss:0.013643580488860607  gen_loss:4.512739181518555 recA Loss:1.0503499507904053 recB Loss:0.05891481041908264
epoch:1  disc_loss:0.016882285475730896  gen_loss:4.7121381759643555 recA Loss:1.0291489362716675 recB Loss:0.05358307063579559
epoch:1  disc_loss:0.01676405593752861  gen_loss:5.777318000793457 recA Loss:1.0256309509277344 recB Loss:0.09083949029445648
epoch:1  disc_loss:0.01353235449641943  gen_loss:4.69516658782959 recA Loss:1.0334796905517578 recB Loss:0.05808669328689575
epoch:2  disc_loss:0.013784321956336498  gen_loss:4.6128129959106445 recA Loss:1.0272371768951416 recB Loss:0.0655679702758789
epoch:2  disc_loss:0.012395376339554787  gen_loss:4.569788932800293 recA Loss:1.0166682004928589 recB Loss:0.06165506690740585
epoch:2  disc_loss:0.012934714555740356  gen_loss:5.0562872886657715 recA Loss:1.0234354734420776 recB Loss:0.07715937495231628
epoch:2  disc_loss:0.010235204361379147  gen_loss:4.800635814666748 recA Loss:1.0210286378860474 recB Loss:0.056

In [ ]:
dis_fake_labB.shape

In [23]:
rec_errorB

tensor(109.0262, device='cuda:2', grad_fn=<L1LossBackward>)

In [1]:
!nvidia-smi

Sun Dec 22 21:39:49 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.129      Driver Version: 410.129      CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P6            On   | 00000000:17:00.0 Off |                    0 |
| N/A   35C    P0    77W /  90W |  14231MiB / 15261MiB |     96%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P6            On   | 00000000:45:00.0 Off |                    0 |
| N/A   36C    P0    92W /  90W |   9918MiB / 15261MiB |     99%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

TypeError: __call__() missing 1 required positional argument: 'img'